<a href="https://colab.research.google.com/github/unicamp-dl/IA025_2022S1/blob/main/Final_project/Karen_Rosero/Fine_tuning_wav2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sound detection and classification using transformers
## Fine-tuning notebook

## Projeto final da disclipina IA025
## Autora: Karen Rosero

Instalo as bibliotecas faltantes

In [ ]:
#!pip install transformers

In [ ]:
#!pip install ipywidgets

In [ ]:
#!pip install pytorch-lightning

In [ ]:
#!pip install nvidia-ml-py3

In [ ]:
#!pip install neptune-client

In [ ]:
#!pip install lightning-bolts

In [ ]:
!pip install torchmetrics

Defaulting to user installation because normal site-packages is not writeable


Importo as bibliotecas necessárias

In [ ]:
from transformers import Wav2Vec2Processor, Wav2Vec2Model, Wav2Vec2Config
import torch
from transformers.models.wav2vec2.modeling_wav2vec2 import Wav2Vec2FeatureEncoder, Wav2Vec2NoLayerNormConvLayer, Wav2Vec2LayerNormConvLayer
from torch import nn
from transformers.activations import ACT2FN
import ipywidgets 
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import csv
import torchaudio
import pytorch_lightning as pl
import nvidia_smi
#from pytorch_lightning.loggers.neptune import NeptuneLogger
from pytorch_lightning.loggers import NeptuneLogger
from IPython.display import display, HTML
from dataclasses import dataclass, field
from torch.utils.data import DataLoader
from typing import Any, Dict, List, Optional, Union
# usadas para as métricas
from pl_bolts.optimizers.lr_scheduler import LinearWarmupCosineAnnealingLR
from torchmetrics import Accuracy
from torchmetrics import F1Score

GPU a ser utilizada

In [ ]:
!nvidia-smi

Wed Jun 29 16:46:36 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.91.03    Driver Version: 460.91.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  TITAN V             Off  | 00000000:02:00.0  On |                  N/A |
| 33%   48C    P8    30W / 250W |    367MiB / 12063MiB |      2%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
print(f"Pytorch Lightning Version: {pl.__version__}")
nvidia_smi.nvmlInit()
handle = nvidia_smi.nvmlDeviceGetHandleByIndex(0)
print(f"Device name: {nvidia_smi.nvmlDeviceGetName(handle)}")

Pytorch Lightning Version: 1.6.4
Device name: b'TITAN V'


Defino hiperparâmetros

In [ ]:
version = "wav2vec2-sound_detection_test1" #@param {type: "string"}
lr = 1e-5#@param {type: "number"}
w_decay = 0#@param {type: "number"}
bs = 64#@param {type: "integer"}
accum_grads = 1#@param {type: "integer"}
patience = 30#@param {type: "integer"}
max_epochs = 200#@param {type: "integer"}
# warmup_steps = 1000#@param {type: "integer"}
hold_epochs = 20#@param {type: "integer"}
pretrained = "facebook/wav2vec2-base-960h"#@param {type: "string"}
wav2vec2_processor = "facebook/wav2vec2-base-960h"#@param {type: "string"}
freeze_finetune_updates = 10000#@param {type: "integer"}
warm_up_steps = 20000#@param {type: "integer"}
apply_mask=True#@param {type: "boolean"}
mask_time_length= 1#@param {type: "integer"}

# Define hyperparameters
hparams = {"version": version,
          "lr": lr,
          "w_decay": w_decay,
          "bs": bs,
          "patience": patience,
          "hold_epochs":hold_epochs,
          "accum_grads": accum_grads,
          "pretrained":pretrained,
          "wav2vec2_processor": wav2vec2_processor,
          "freeze_finetune_updates":freeze_finetune_updates,
          "warm_up_steps":warm_up_steps,
          "apply_mask":apply_mask, 
          "mask_time_length":mask_time_length, 
          "max_epochs": max_epochs}
hparams

{'version': 'wav2vec2-sound_detection_test1',
 'lr': 1e-05,
 'w_decay': 0,
 'bs': 64,
 'patience': 30,
 'hold_epochs': 20,
 'accum_grads': 1,
 'pretrained': 'facebook/wav2vec2-base-960h',
 'wav2vec2_processor': 'facebook/wav2vec2-base-960h',
 'freeze_finetune_updates': 10000,
 'warm_up_steps': 20000,
 'apply_mask': True,
 'mask_time_length': 1,
 'max_epochs': 200}

Crio o processador e modelo originais do wav2vec2

In [ ]:
processor = Wav2Vec2Processor.from_pretrained(hparams["wav2vec2_processor"])
model = Wav2Vec2Model.from_pretrained(hparams["wav2vec2_processor"])

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2Model: ['lm_head.weight', 'lm_head.bias']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


O modelo original recebe um canal na entrada. Isso precisa ser mudado para 4 canais

In [ ]:
print(model)

Wav2Vec2Model(
  (feature_extractor): Wav2Vec2FeatureEncoder(
    (conv_layers): ModuleList(
      (0): Wav2Vec2GroupNormConvLayer(
        (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
        (activation): GELUActivation()
        (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
      )
      (1): Wav2Vec2NoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        (activation): GELUActivation()
      )
      (2): Wav2Vec2NoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        (activation): GELUActivation()
      )
      (3): Wav2Vec2NoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        (activation): GELUActivation()
      )
      (4): Wav2Vec2NoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        (activation): GELUActivation()
      )
      (5): Wav2Vec2NoLay

### Adaptação do modelo wav2vec2 para receber e processar 4 canais de áudio na entrada

In [ ]:
class Wav2Vec2GroupNormConvLayer(nn.Module):
    def __init__(self, config, layer_id=0):
        super().__init__()
        self.in_conv_dim = config.conv_dim[layer_id - 1] if layer_id > 0 else 4 # define 4 canais na camada de entrada
        self.out_conv_dim = config.conv_dim[layer_id]

        self.conv = nn.Conv1d(
            self.in_conv_dim,
            self.out_conv_dim,
            kernel_size=config.conv_kernel[layer_id],
            stride=config.conv_stride[layer_id],
            bias=config.conv_bias,
        )
        self.activation = ACT2FN[config.feat_extract_activation]

        self.layer_norm = nn.GroupNorm(num_groups=self.out_conv_dim, num_channels=self.out_conv_dim, affine=True)

    def forward(self, hidden_states):
        hidden_states = self.conv(hidden_states)
        hidden_states = self.layer_norm(hidden_states)
        hidden_states = self.activation(hidden_states)
        return hidden_states
    
class Wav2Vec2_4ChannelFeatureEncoder(nn.Module):
    """Construct the features from raw audio waveform"""

    def __init__(self, config):
        super().__init__()

        if config.feat_extract_norm == "group":
            conv_layers = [Wav2Vec2GroupNormConvLayer(config, layer_id=0)] + [
                Wav2Vec2NoLayerNormConvLayer(config, layer_id=i + 1) for i in range(config.num_feat_extract_layers - 1)
            ]
        elif config.feat_extract_norm == "layer":
            conv_layers = [
                Wav2Vec2LayerNormConvLayer(config, layer_id=i) for i in range(config.num_feat_extract_layers)
            ]
        else:
            raise ValueError(
                f"`config.feat_extract_norm` is {config.feat_extract_norm}, but has to be one of ['group', 'layer']"
            )
        self.conv_layers = nn.ModuleList(conv_layers)
        self.gradient_checkpointing = False
        self._requires_grad = True

    def _freeze_parameters(self):
        for param in self.parameters():
            param.requires_grad = False
        self._requires_grad = False

    def forward(self, input_values):
        hidden_states = input_values[:] # mudou para que receba todos os canais (4)
        print("hidden_states", hidden_states.shape)

        # make sure hidden_states require grad for gradient_checkpointing
        if self._requires_grad and self.training:
            hidden_states.requires_grad = True

        for conv_layer in self.conv_layers:
            if self._requires_grad and self.gradient_checkpointing and self.training:

                def create_custom_forward(module):
                    def custom_forward(*inputs):
                        return module(*inputs)

                    return custom_forward

                hidden_states = torch.utils.checkpoint.checkpoint(
                    create_custom_forward(conv_layer),
                    hidden_states,
                )
            else:
                hidden_states = conv_layer(hidden_states)

        return hidden_states    

# Crio o novo modelo que herda os processos de Wav2Vec2, mas usa o extrator de features baseado em 4 canais
class Wav2Vec2_4ChannelModel(Wav2Vec2Model):
    def __init__(self, config: Wav2Vec2Config):
        super().__init__(config)

        # del self.feature_extractor
        self.feature_extractor = Wav2Vec2_4ChannelFeatureEncoder(config)    

In [ ]:
model4c = Wav2Vec2_4ChannelModel.from_pretrained("facebook/wav2vec2-base-960h", ignore_mismatched_sizes=True)

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2_4ChannelModel: ['lm_head.weight', 'lm_head.bias']
- This IS expected if you are initializing Wav2Vec2_4ChannelModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2_4ChannelModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2_4ChannelModel were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of Wav2Vec2_4ChannelModel were not initialized from the model c

Agora a primeira camada convolucional do modelo recebe 4 canais como entrada

In [ ]:
print(model4c)

Wav2Vec2_4ChannelModel(
  (feature_extractor): Wav2Vec2_4ChannelFeatureEncoder(
    (conv_layers): ModuleList(
      (0): Wav2Vec2GroupNormConvLayer(
        (conv): Conv1d(4, 512, kernel_size=(10,), stride=(5,), bias=False)
        (activation): GELUActivation()
        (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
      )
      (1): Wav2Vec2NoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        (activation): GELUActivation()
      )
      (2): Wav2Vec2NoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        (activation): GELUActivation()
      )
      (3): Wav2Vec2NoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        (activation): GELUActivation()
      )
      (4): Wav2Vec2NoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        (activation): GELUActivation()
      )
      

### Dataloader da base de dados ANSYN (TUT Sound Events 2018 - Ambisonic, Anechoic and Synthetic Impulse Response Dataset)

In [ ]:
audio_path = '/home/lab_acustica/Documentos/ANSYN_Dataset/audio_200ms_ov1/'
labels_path = '/home/lab_acustica/Documentos/ANSYN_Dataset/label_200ms_ov1/'

In [ ]:
train_total_list = []
X_test = []
for file in os.listdir(audio_path):
    if 'tra' in file:
        train_total_list.append(file[:-4])
    elif 'tst' in file:
        X_test.append(file[:-4])

In [ ]:
train_set, val_set = torch.utils.data.random_split(train_total_list, [int(len(train_total_list)*0.7), int(len(train_total_list)*0.3)])

In [ ]:
X_train, X_val = train_test_split(train_total_list,  test_size=0.3, random_state=42, shuffle = True)
print('Treinamento',len(X_train), ',  Validação ',len(X_val), ',  Teste',len(X_test))

Treinamento 75600 ,  Validação  32400 ,  Teste 27000


In [ ]:
test_classes = []
for i in range(len(X_test)):    
    if os.path.getsize(labels_path+str(X_test[i])+'.csv')==0:
        test_classes.append(str(-1))
    else:
        label_file = csv.reader(open(labels_path+str(X_test[i])+'.csv', 'r'))
        for count , line in enumerate(label_file):
            test_classes.append(line[0])
print(len(test_classes))

val_classes = []
for i in range(len(X_val)):    
    if os.path.getsize(labels_path+X_val[i]+'.csv')==0:
        val_classes.append(str(-1))
    else:
        label_file = csv.reader(open(labels_path+X_val[i]+'.csv', 'r'))
        for count , line in enumerate(label_file):
            val_classes.append(line[0])
print(len(val_classes))

train_classes = []
for i in range(len(X_train)):    
    if os.path.getsize(labels_path+X_train[i]+'.csv')==0:
        train_classes.append(str(-1))
    else:
        label_file = csv.reader(open(labels_path+X_train[i]+'.csv', 'r'))
        for count , line in enumerate(label_file):
            train_classes.append(line[0])
print(len(train_classes))

27000
32400
75600


In [ ]:
d = {'Filename': X_train, 'Class': train_classes}
df_train = pd.DataFrame(data=d)
print(df_train.shape)
df_train.head()

(75600, 2)


,Filename,Class
0,tra_045_ov1_s3_106,4
1,tra_165_ov1_s2_17,0
2,tra_102_ov1_s1_139,2
3,tra_105_ov1_s2_131,4
4,tra_023_ov1_s3_90,0


In [ ]:
df_val = pd.DataFrame(data={'Filename': X_val, 'Class': val_classes})
print(df_val.shape)
df_val.head()

(32400, 2)


,Filename,Class
0,tra_065_ov1_s1_1,1
1,tra_096_ov1_s3_1,0
2,tra_126_ov1_s1_9,1
3,tra_067_ov1_s1_61,3
4,tra_115_ov1_s3_56,1


In [ ]:
df_test = pd.DataFrame(data={'Filename': X_test, 'Class': test_classes})
print(df_test.shape)
df_test.head()

(27000, 2)


,Filename,Class
0,tst_032_ov1_s2_5,7
1,tst_051_ov1_s1_45,-1
2,tst_026_ov1_s2_64,5
3,tst_027_ov1_s3_53,3
4,tst_005_ov1_s2_44,2


In [ ]:
class ANSYN_Dataset_20ms(torch.utils.data.Dataset):

    def __init__(self, filenames):    
        self.filenames = filenames
        self.audio_path = "/home/lab_acustica/Documentos/ANSYN_Dataset/audio_200ms_ov1/"
    
    def normalize_layer(self, feats):    # função para normalizar
        with torch.no_grad():
            feats = torch.nn.functional.layer_norm(feats, feats.shape)
        return feats

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, index):

        feats, _ = torchaudio.load(self.audio_path + self.filenames['Filename'][index]+'.wav')
        feats = self.normalize_layer(feats)
        target = torch.tensor(int(self.filenames['Class'][index])).long()
        return {"input_values": feats, "target":target}

In [ ]:
train_dataset = ANSYN_Dataset_20ms(df_train)                           
val_dataset =  ANSYN_Dataset_20ms(df_val)  
test_dataset = ANSYN_Dataset_20ms(df_test)

In [ ]:
print('Número de amostras de treinamento:', len(train_dataset))
print('Número de amostras de validação:', len(val_dataset))
print('Número de amostras de teste:', len(test_dataset))

Número de amostras de treinamento: 75600
Número de amostras de validação: 32400
Número de amostras de teste: 27000


In [ ]:
print(train_dataset[0]["input_values"].shape, 'Class ', train_dataset[0]["target"])


torch.Size([4, 3200]) Class  tensor(4)


In [ ]:
target_sampling_rate = processor.feature_extractor.sampling_rate
print(f"Target sampling rate: {target_sampling_rate}")

inputs = processor(train_dataset[0]["input_values"], sampling_rate=target_sampling_rate, return_tensors="pt")
print(inputs)
print(inputs["input_values"].shape)

Target sampling rate: 16000
{'input_values': tensor([[[ 0.0263, -0.4117,  0.8765,  ..., -0.0401, -0.0114,  0.1829],
         [-0.0035, -0.0027,  0.0431,  ..., -0.0243, -0.0223, -0.0099],
         [-0.0161,  0.5285, -0.8872,  ...,  0.0451,  0.0300, -0.2385],
         [ 0.0251, -0.5538,  1.1259,  ..., -0.0488, -0.0898,  0.3146]]])}
torch.Size([1, 4, 3200])


In [ ]:
with torch.no_grad():
    outputs = model4c(**inputs)

last_hidden_states = outputs.last_hidden_state
print(last_hidden_states.shape)

hidden_states torch.Size([1, 4, 3200])
torch.Size([1, 9, 768])


In [ ]:
@dataclass
class DataCollatorWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["target"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        #print('batch', batch)
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=True,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )
        #print('labels_batch', labels_batch)


        labels = labels_batch["input_ids"]

        batch["target"] = labels

        return batch

In [ ]:
data_collator = DataCollatorWithPadding(processor=processor,
                                        # max_length=188,
                                        padding=False)

In [ ]:
batch_size = hparams["bs"]

train_dataloader = DataLoader(train_dataset, batch_size=batch_size,
                              collate_fn = data_collator,
                              shuffle=True, num_workers=4)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size,
                            collate_fn = data_collator,
                            shuffle=False, num_workers=4)

print('Número de minibatches de treinamento:', len(train_dataloader))
print('Número de minibatches de validação:', len(val_dataloader))

batch = next(iter(train_dataloader))
print(batch)

x_train, y_train = batch['input_values'], batch['target']
print("\nDimensões dos dados de um minibatch - Audio:", x_train.size())
# print("\nDimensões dos dados de um minibatch:", padding_mask.size())
print("\nDimensões dos dados de um minibatch - Target:", y_train.size())
print("Valores mínimo e máximo entrada: ", torch.min(x_train), torch.max(x_train))
print("Valores mínimo e máximo saída: ", torch.min(y_train), torch.max(y_train))
print("Tipo dos dados dos áudios:         ", type(x_train))
print("Tipo das classes das classes:       ", type(y_train))

Número de minibatches de treinamento: 1182
Número de minibatches de validação: 507
{'input_values': tensor([[[-2.9669e-01, -4.7600e-01, -1.6874e-01,  ...,  2.5148e-02,
           3.3662e-02,  4.3271e-02],
         [-3.2242e-01, -5.1529e-01, -1.3612e-01,  ..., -3.0559e-02,
          -7.3276e-03, -4.5722e-02],
         [ 3.0519e-01,  5.0594e-01,  2.1356e-01,  ...,  4.9681e-02,
           2.6925e-02,  5.6713e-02],
         [ 2.5903e-01,  3.7102e-01,  9.2295e-02,  ...,  3.6491e-02,
           3.4278e-02,  4.6662e-02]],

        [[-1.6944e-01, -4.9374e-01, -3.8197e-01,  ..., -7.5608e-01,
          -3.9245e-01, -6.1471e-01],
         [ 9.8331e-02,  3.1871e-01,  4.7729e-01,  ...,  3.5419e-01,
           4.5227e-01,  3.1557e-01],
         [-1.0776e-02, -6.8585e-01, -6.8530e-01,  ..., -8.4598e-01,
          -5.2554e-01, -6.0332e-01],
         [ 5.6402e-02,  4.8093e-01,  5.4826e-01,  ...,  7.9382e-01,
           7.7527e-01,  7.2325e-01]],

        [[-6.4321e-03, -5.1480e-03,  2.8070e-02,  ...,  

Defino as métricas que serão usadas

In [ ]:
n_classes = 12
f1 = F1Score(num_classes=n_classes, average='micro')
accuracy = Accuracy(num_classes=n_classes)

### Criando o  lightning module

In [ ]:
class Wav2Vec2_sound_detection(pl.LightningModule):
    def __init__(self, *args, **kwargs):
        super().__init__()

        self.hparams.update(hparams)

        self.freeze_finetune_updates = hparams["freeze_finetune_updates"]
        
        self.model = model4c
        #self.model = Wav2Vec2_sound_detection.from_pretrained(hparams["pretrained"],
                                                 #conv_stride=[5,1,1,1,1,1,1],
                                                 #apply_spec_augment=hparams["apply_mask"],
                                                 #mask_time_length=hparams["mask_time_length"])
        # self.model.feature_extractor._freeze_parameters()
        
        # freeze base-model
        # for param in self.model.parameters():
        #     param.requires_grad = False
        
        self.projector = nn.Linear(self.model.config.hidden_size, self.model.config.classifier_proj_size)
        self.final_layer = nn.Linear(self.model.config.classifier_proj_size, n_classes)    
    
    def forward(self, samples):
        
        # print("input_values", samples["input_values"].shape)
        # print("attention_mask", samples["attention_mask"].shape)

        ft = self.freeze_finetune_updates <= self.trainer.global_step
       
        with torch.no_grad() if not ft else contextlib.ExitStack():
              hidden_states = self.model(**samples).last_hidden_state
        
        # print('hidden_states', hidden_states.shape)
        # print('hidden_states squeeze', hidden_states[:,0,:].shape)

        padding_mask = self.model._get_feature_vector_attention_mask(hidden_states.shape[1], samples["attention_mask"])
        # print("padding_mask", padding_mask.shape)

        hidden_states[~padding_mask] = 0.0
        
        pooled_output = hidden_states.sum(dim=1) / padding_mask.sum(dim=1).view(-1, 1)

        # print("pooled_output", pooled_output.shape)

        proj_pooled = self.projector(pooled_output)
        
        # print("proj_pooled", proj_pooled.shape)

        preds = self.final_layer(proj_pooled)
        
        return F.log_softmax(preds, dim=1)
    
    def _get_feature_vector_attention_mask(self, feature_vector_length: int, attention_mask: torch.LongTensor):
        output_lengths = self._get_feat_extract_output_lengths(attention_mask.sum(-1)).to(torch.long)
        batch_size = attention_mask.shape[0]

        attention_mask = torch.zeros(
            (batch_size, feature_vector_length), dtype=attention_mask.dtype, device=attention_mask.device
        )
        # these two operations makes sure that all values before the output lengths idxs are attended to
        attention_mask[(torch.arange(attention_mask.shape[0], device=attention_mask.device), output_lengths - 1)] = 1
        attention_mask = attention_mask.flip([-1]).cumsum(-1).flip([-1]).bool()
        return attention_mask


    def training_step(self, train_batch, batch_idx):
        
        y_value = train_batch.pop("target")
        log_softs = self.forward(train_batch)

        # print('train preds:', preds) 
        # print('train y_value:', y_value)

        # loss mse or rmspe compute
        # mse_loss = F.mse_loss(preds, y_value)
        loss = F.nll_loss(log_softs, y_value)

        # print('train mse', mse_loss)
        # print('#########################')

        # self.log('mse_loss_step', loss, on_step=True, prog_bar=True)
        self.log('loss_step', loss, on_step=True, prog_bar=True)
        
        return loss

    def training_epoch_end(self, outputs):
        loss = torch.stack([x['loss'] for x in outputs]).mean()       

        self.log("train_loss", loss, prog_bar=True)
  
    def validation_step(self, val_batch, batch_idx):
        
        y_value = val_batch.pop("target")

        # predict 
        log_softs = self.forward(val_batch)
        preds = torch.argmax(log_softs, dim=1) 
        
        # print('log_softs:', log_softs.shape) 
        # print('preds:', preds.shape) 
        # print('y_value:', y_value.shape)
        
        # print('val preds:', preds) 
        # print('val y_value:', y_value)       

        val_acc = accuracy(preds.cpu(), y_value.cpu())
        val_f1 = f1(preds.cpu(), y_value.cpu())
        val_loss = F.nll_loss(log_softs, y_value)

        # print('val mse', mse_loss)
        # print('val rmspe', rmspe_loss)
        # print('----------------------')

        self.log('val_acc', val_acc, prog_bar=True)
        self.log('val_f1', val_f1, prog_bar=True)
        self.log('val_loss', val_loss, prog_bar=True)

        return {"val_acc_step": val_acc, "val_f1_step": val_f1, "val_loss_step": val_loss}

    def validation_epoch_end(self, outputs):
        acc_mean = torch.stack([x['val_acc_step'] for x in outputs]).mean()
        f1_mean = torch.stack([x['val_f1_step'] for x in outputs]).mean()
        loss_mean = torch.stack([x['val_loss_step'] for x in outputs]).mean()

        self.log("val_acc", acc_mean, prog_bar=True)
        self.log("val_f1", f1_mean, prog_bar=True)
        self.log("val_loss", loss_mean, prog_bar=True)
  
    def test_step(self, test_batch, batch_idx):
        
        y_value = test_batch.pop("target")

        # predict 
        log_softs = self.forward(test_batch)
        preds = torch.argmax(log_softs, dim=1) 

        test_acc = accuracy(preds.cpu(), y_value.cpu())
        test_f1 = f1(preds.cpu(), y_value.cpu())
        test_loss = F.nll_loss(log_softs, y_value)
        
        self.log('test_acc', test_acc, prog_bar=True)
        self.log('test_f1', test_f1, prog_bar=True)
        self.log('test_loss', test_loss, prog_bar=True)

        return {"test_acc_step": test_acc, "test_f1_step": test_f1,  "test_loss_step": test_loss}

    def test_epoch_end(self, outputs):
        acc_mean = torch.stack([x['test_acc_step'] for x in outputs]).mean()
        f1_mean = torch.stack([x['test_f1_step'] for x in outputs]).mean()
        loss_mean = torch.stack([x['test_loss_step'] for x in outputs]).mean()

        self.log("test_acc", acc_mean, prog_bar=True)
        self.log("test_f1", f1_mean, prog_bar=True)
        self.log("test_loss", loss_mean, prog_bar=True)
    
    ## learning rate warm-up
    def optimizer_step(
        self,
        epoch,
        batch_idx,
        optimizer,
        optimizer_idx,
        optimizer_closure,
        on_tpu=False,
        using_native_amp=False,
        using_lbfgs=False,
    ):
        # skip the first 2000 steps
        if self.trainer.global_step < hparams["warm_up_steps"]:
            lr_scale = min(1.0, float(self.trainer.global_step + 1) / hparams["warm_up_steps"])
            for pg in optimizer.param_groups:
                pg["lr"] = lr_scale * self.hparams["lr"]

        # update params
        optimizer.step(closure=optimizer_closure)

    def configure_optimizers(self):

        optimizer = torch.optim.Adam(self.parameters(),
                         lr=self.hparams["lr"],
                         betas=(0.9,0.98),
                         eps=1e-6,
                         weight_decay=self.hparams["w_decay"])
        
        scheduler = LinearWarmupCosineAnnealingLR(optimizer, 
                                                  eta_min=0, # final-lr
                                                  warmup_start_lr=self.hparams["lr"],
                                                  warmup_epochs=self.hparams["hold_epochs"], # hold_epochs
                                                  max_epochs=self.hparams["max_epochs"])
        
        return {'optimizer': optimizer, 'lr_scheduler': scheduler}